# A2
* GCR = 0.4
* Albedo = 0.5
* Hub Height = 1.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* A GCR of 0.4 for the Trina Vertex module (length = 2.384m) corresponds to a rowto-row spacing or pitch of 5.96m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/A2'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/A2


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+245.gd0d9c92.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'A2'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 5.96 # m
albedo = 0.5
hub_height = 1.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/A2
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
Loading albedo, 1 value(s), 0.500 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json


In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-01-01_0000', endtime='2022-02-01_0000')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~307 skyfiles. 
Created 304 skyfiles in /skies/


{'2022-01-01_0830': {'surf_azm': 90.0,
  'surf_tilt': 53.19,
  'theta': -53.19,
  'dni': 0,
  'ghi': 78,
  'dhi': 76,
  'temp_air': 3.7,
  'wind_speed': 5.2,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_0830.rad'},
 '2022-01-01_0930': {'surf_azm': 90.0,
  'surf_tilt': 50.17,
  'theta': -50.17,
  'dni': 0,
  'ghi': 85,
  'dhi': 82,
  'temp_air': 3.3,
  'wind_speed': 5.0,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_0930.rad'},
 '2022-01-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 29.16,
  'theta': -29.16,
  'dni': 5,
  'ghi': 112,
  'dhi': 108,
  'temp_air': 3.9,
  'wind_speed': 4.7,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_1030.rad'},
 '2022-01-01_1130': {'surf_azm': 90.0,
  'surf_tilt': 4.27,
  'theta': -4.27,
  'dni': 60,
  'ghi': 265,
  'dhi': 232,
  'temp_air': 3.3,
  'wind_speed': 5.5,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-01-01_1130.rad'},
 '2022-01-01_1230': {'surf_azm': 270.0,
  'surf_tilt': 21.33,
  'theta': 21.33,
  'dni': 0,
  'ghi': 121,
  'dhi': 11

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~304 .rad files for gendaylit 1-axis workflow (this takes a minute..)
304 Radfiles created in /objects/

Making 304 octfiles in root directory.
Created 1axis_2022-01-01_0830.oct
Created 1axis_2022-01-01_0930.oct
Created 1axis_2022-01-01_1030.oct
Created 1axis_2022-01-01_1130.oct
Created 1axis_2022-01-01_1230.oct
Created 1axis_2022-01-01_1330.oct
Created 1axis_2022-01-01_1430.oct
Created 1axis_2022-01-01_1530.oct
Created 1axis_2022-01-01_1630.oct
Created 1axis_2022-01-02_0830.oct
Created 1axis_2022-01-02_0930.oct
Created 1axis_2022-01-02_1030.oct
Created 1axis_2022-01-02_1130.oct
Created 1axis_2022-01-02_1230.oct
Created 1axis_2022-01-02_1330.oct
Created 1axis_2022-01-02_1430.oct
Created 1axis_2022-01-02_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2161: Warning: Warning: positive Perez parameter B (= 0.186245), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-01-02_1630.oct
Created 1axis_2022-01-03_0830.oct
Created 1axis_2022-01-03_0930.oct
Created 1axis_2022-01-03_1030.oct
Created 1axis_2022-01-03_1130.oct
Created 1axis_2022-01-03_1230.oct
Created 1axis_2022-01-03_1330.oct
Created 1axis_2022-01-03_1430.oct
Created 1axis_2022-01-03_1530.oct


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2161: Warning: Warning: positive Perez parameter B (= 0.117978), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-01-03_1630.oct
Created 1axis_2022-01-04_0730.oct
Created 1axis_2022-01-04_0830.oct
Created 1axis_2022-01-04_0930.oct
Created 1axis_2022-01-04_1030.oct
Created 1axis_2022-01-04_1130.oct
Created 1axis_2022-01-04_1230.oct
Created 1axis_2022-01-04_1330.oct
Created 1axis_2022-01-04_1430.oct
Created 1axis_2022-01-04_1530.oct


...

Created 1axis_2022-01-20_1630.oct
Created 1axis_2022-01-21_0730.oct
Created 1axis_2022-01-21_0830.oct
Created 1axis_2022-01-21_0930.oct
Created 1axis_2022-01-21_1030.oct
Created 1axis_2022-01-21_1130.oct
Created 1axis_2022-01-21_1230.oct
Created 1axis_2022-01-21_1330.oct
Created 1axis_2022-01-21_1430.oct
Created 1axis_2022-01-21_1530.oct
Created 1axis_2022-01-21_1630.oct
Created 1axis_2022-01-22_0730.oct
Created 1axis_2022-01-22_0830.oct
Created 1axis_2022-01-22_0930.oct
Created 1axis_2022-01-22_1030.oct
Created 1axis_2022-01-22_1130.oct
Created 1axis_2022-01-22_1230.oct
Created 1axis_2022-01-22_1330.oct
Created 1axis_2022-01-22_1430.oct
Created 1axis_2022-01-22_1530.oct
Created 1axis_2022-01-22_1630.oct
Created 1axis_2022-01-23_0730.oct
Created 1axis_2022-01-23_0830.oct
Created 1axis_2022-01-23_0930.oct
Created 1axis_2022-01-23_1030.oct
Created 1axis_2022-01-23_1130.oct
Created 1axis_2022-01-23_1230.oct
Created 1axis_2022-01-23_1330.oct
Created 1axis_2022-01-23_1430.oct
Created 1axis_

/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2161: Warning: Warning: positive Perez parameter B (= 0.014182), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-01-23_1630.oct
Created 1axis_2022-01-24_0730.oct
Created 1axis_2022-01-24_0830.oct
Created 1axis_2022-01-24_0930.oct
Created 1axis_2022-01-24_1030.oct
Created 1axis_2022-01-24_1130.oct
Created 1axis_2022-01-24_1230.oct
Created 1axis_2022-01-24_1330.oct
Created 1axis_2022-01-24_1430.oct
Created 1axis_2022-01-24_1530.oct
Created 1axis_2022-01-24_1630.oct
Created 1axis_2022-01-25_0730.oct
Created 1axis_2022-01-25_0830.oct
Created 1axis_2022-01-25_0930.oct
Created 1axis_2022-01-25_1030.oct
Created 1axis_2022-01-25_1130.oct
Created 1axis_2022-01-25_1230.oct
Created 1axis_2022-01-25_1330.oct
Created 1axis_2022-01-25_1430.oct
Created 1axis_2022-01-25_1530.oct
Created 1axis_2022-01-25_1630.oct
Created 1axis_2022-01-26_0730.oct
Created 1axis_2022-01-26_0830.oct
Created 1axis_2022-01-26_0930.oct
Created 1axis_2022-01-26_1030.oct
Created 1axis_2022-01-26_1130.oct
Created 1axis_2022-01-26_1230.oct
Created 1axis_2022-01-26_1330.oct
Created 1axis_2022-01-26_1430.oct
Created 1axis_

/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py:2161: Warning: Warning: positive Perez parameter B (= 0.019917), printing error sky
  warnings.warn(err[9:], Warning)


Created 1axis_2022-01-27_1630.oct
Created 1axis_2022-01-28_0730.oct
Created 1axis_2022-01-28_0830.oct
Created 1axis_2022-01-28_0930.oct
Created 1axis_2022-01-28_1030.oct
Created 1axis_2022-01-28_1130.oct
Created 1axis_2022-01-28_1230.oct
Created 1axis_2022-01-28_1330.oct
Created 1axis_2022-01-28_1430.oct
Created 1axis_2022-01-28_1530.oct
Created 1axis_2022-01-28_1630.oct
Created 1axis_2022-01-29_0730.oct
Created 1axis_2022-01-29_0830.oct
Created 1axis_2022-01-29_0930.oct
Created 1axis_2022-01-29_1030.oct
Created 1axis_2022-01-29_1130.oct
Created 1axis_2022-01-29_1230.oct
Created 1axis_2022-01-29_1330.oct
Created 1axis_2022-01-29_1430.oct
Created 1axis_2022-01-29_1530.oct
Created 1axis_2022-01-29_1630.oct
Created 1axis_2022-01-30_0730.oct
Created 1axis_2022-01-30_0830.oct
Created 1axis_2022-01-30_0930.oct
Created 1axis_2022-01-30_1030.oct
Created 1axis_2022-01-30_1130.oct
Created 1axis_2022-01-30_1230.oct
Created 1axis_2022-01-30_1330.oct
Created 1axis_2022-01-30_1430.oct
Created 1axis_

In [14]:
#trackerdict = demo.analysis1axis(sensorsy=2) # Middle Module

In [15]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

Linescan in process: 1axis_2022-01-01_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-01-01_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-01_0830_Row3_Module1.csv
Index: 2022-01-01_0830. Wm2Front: 64.38175333333334. Wm2Back: 29.92001666666667
Linescan in process: 1axis_2022-01-01_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-01-01_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-01_0930_Row3_Module1.csv
Index: 2022-01-01_0930. Wm2Front: 68.390705. Wm2Back: 31.850725000000004
Linescan in process: 1axis_2022-01-01_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-01_1030_Row3_Module1.csv
Index: 2022-01-01_1030. Wm2Front: 103.98243333333333. Wm2Back: 38.85775
Linescan in process: 1axis_2022-01-01_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-01-01_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-01_1130_Row3_Module1.csv
Index: 2022-01-01_1130. Wm2Front: 262.4621166666667

Linescan in process: 1axis_2022-01-04_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-04_1130_Row3_Module1.csv
Index: 2022-01-04_1130. Wm2Front: 589.9944333333333. Wm2Back: 269.41746666666666
Linescan in process: 1axis_2022-01-04_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-01-04_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-04_1230_Row3_Module1.csv
Index: 2022-01-04_1230. Wm2Front: 615.7387000000001. Wm2Back: 253.22198333333336
Linescan in process: 1axis_2022-01-04_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-01-04_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-04_1330_Row3_Module1.csv
Index: 2022-01-04_1330. Wm2Front: 689.9897500000001. Wm2Back: 187.82371666666666
Linescan in process: 1axis_2022-01-04_1430_Row3_Module1_Front
Linescan in process: 1axis_2022-01-04_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-04_1430_Row3_Module1.csv
Index: 2022-01-04_1430. Wm2Front: 741.7607166666667. Wm2Back: 122.26418333333334
Linescan in pr

Linescan in process: 1axis_2022-01-07_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-07_1230_Row3_Module1.csv
Index: 2022-01-07_1230. Wm2Front: 607.3965833333333. Wm2Back: 250.92638333333332
Linescan in process: 1axis_2022-01-07_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-01-07_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-07_1330_Row3_Module1.csv
Index: 2022-01-07_1330. Wm2Front: 676.3910333333333. Wm2Back: 186.02811666666668
Linescan in process: 1axis_2022-01-07_1430_Row3_Module1_Front
Linescan in process: 1axis_2022-01-07_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-07_1430_Row3_Module1.csv
Index: 2022-01-07_1430. Wm2Front: 734.1423499999999. Wm2Back: 121.68700000000001
Linescan in process: 1axis_2022-01-07_1530_Row3_Module1_Front
Linescan in process: 1axis_2022-01-07_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-07_1530_Row3_Module1.csv
Index: 2022-01-07_1530. Wm2Front: 449.97065. Wm2Back: 85.44561
Linescan in process: 1axis_2022-

Saved: results/irr_1axis_2022-01-10_1230_Row3_Module1.csv
Index: 2022-01-10_1230. Wm2Front: 628.5616333333334. Wm2Back: 260.4463666666667
Linescan in process: 1axis_2022-01-10_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-01-10_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-10_1330_Row3_Module1.csv
Index: 2022-01-10_1330. Wm2Front: 697.5208666666666. Wm2Back: 195.10941666666668
Linescan in process: 1axis_2022-01-10_1430_Row3_Module1_Front
Linescan in process: 1axis_2022-01-10_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-10_1430_Row3_Module1.csv
Index: 2022-01-10_1430. Wm2Front: 751.3150666666667. Wm2Back: 128.41206666666668
Linescan in process: 1axis_2022-01-10_1530_Row3_Module1_Front
Linescan in process: 1axis_2022-01-10_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-10_1530_Row3_Module1.csv
Index: 2022-01-10_1530. Wm2Front: 483.08731666666665. Wm2Back: 91.872425
Linescan in process: 1axis_2022-01-10_1630_Row3_Module1_Front
message: Warning: posi

Linescan in process: 1axis_2022-01-13_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-13_1330_Row3_Module1.csv
Index: 2022-01-13_1330. Wm2Front: 702.6229833333333. Wm2Back: 199.13916666666665
Linescan in process: 1axis_2022-01-13_1430_Row3_Module1_Front
Linescan in process: 1axis_2022-01-13_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-13_1430_Row3_Module1.csv
Index: 2022-01-13_1430. Wm2Front: 762.2087833333333. Wm2Back: 130.88569999999999
Linescan in process: 1axis_2022-01-13_1530_Row3_Module1_Front
Linescan in process: 1axis_2022-01-13_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-13_1530_Row3_Module1.csv
Index: 2022-01-13_1530. Wm2Front: 505.89874999999995. Wm2Back: 94.26427666666666
Linescan in process: 1axis_2022-01-13_1630_Row3_Module1_Front
message: Warning: positive Perez parameter B (= 0.096369), printing error sky
Linescan in process: 1axis_2022-01-13_1630_Row3_Module1_Back
message: Warning: positive Perez parameter B (= 0.096369), printing error s

Saved: results/irr_1axis_2022-01-16_1330_Row3_Module1.csv
Index: 2022-01-16_1330. Wm2Front: 716.565. Wm2Back: 206.88058333333333
Linescan in process: 1axis_2022-01-16_1430_Row3_Module1_Front
Linescan in process: 1axis_2022-01-16_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-16_1430_Row3_Module1.csv
Index: 2022-01-16_1430. Wm2Front: 792.0913666666667. Wm2Back: 137.25556666666665
Linescan in process: 1axis_2022-01-16_1530_Row3_Module1_Front
Linescan in process: 1axis_2022-01-16_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-16_1530_Row3_Module1.csv
Index: 2022-01-16_1530. Wm2Front: 521.0863166666667. Wm2Back: 96.75523166666667
Linescan in process: 1axis_2022-01-16_1630_Row3_Module1_Front
message: Warning: positive Perez parameter B (= 0.026306), printing error sky
Linescan in process: 1axis_2022-01-16_1630_Row3_Module1_Back
message: Warning: positive Perez parameter B (= 0.026306), printing error sky
Saved: results/irr_1axis_2022-01-16_1630_Row3_Module1.csv
Index: 2022

Linescan in process: 1axis_2022-01-19_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-19_1530_Row3_Module1.csv
Index: 2022-01-19_1530. Wm2Front: 165.80058333333335. Wm2Back: 51.99214833333333
Linescan in process: 1axis_2022-01-19_1630_Row3_Module1_Front
Linescan in process: 1axis_2022-01-19_1630_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-19_1630_Row3_Module1.csv
Index: 2022-01-19_1630. Wm2Front: 79.44490333333334. Wm2Back: 27.35159166666667
Linescan in process: 1axis_2022-01-20_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-01-20_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-20_0730_Row3_Module1.csv
Index: 2022-01-20_0730. Wm2Front: 213.9317166666667. Wm2Back: 45.38353666666666
Linescan in process: 1axis_2022-01-20_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-01-20_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-20_0830_Row3_Module1.csv
Index: 2022-01-20_0830. Wm2Front: 527.6350166666666. Wm2Back: 82.55862833333333
Linescan in proce

Linescan in process: 1axis_2022-01-23_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-23_0730_Row3_Module1.csv
Index: 2022-01-23_0730. Wm2Front: 232.49286666666666. Wm2Back: 47.346093333333336
Linescan in process: 1axis_2022-01-23_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-01-23_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-23_0830_Row3_Module1.csv
Index: 2022-01-23_0830. Wm2Front: 635.8586166666666. Wm2Back: 92.23411833333333
Linescan in process: 1axis_2022-01-23_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-01-23_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-23_0930_Row3_Module1.csv
Index: 2022-01-23_0930. Wm2Front: 678.6322833333334. Wm2Back: 151.15966666666668
Linescan in process: 1axis_2022-01-23_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-01-23_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-23_1030_Row3_Module1.csv
Index: 2022-01-23_1030. Wm2Front: 672.2489833333333. Wm2Back: 234.9606
Linescan in process: 1ax

Saved: results/irr_1axis_2022-01-26_0830_Row3_Module1.csv
Index: 2022-01-26_0830. Wm2Front: 178.74046666666666. Wm2Back: 52.26830833333334
Linescan in process: 1axis_2022-01-26_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-01-26_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-26_0930_Row3_Module1.csv
Index: 2022-01-26_0930. Wm2Front: 127.56473333333335. Wm2Back: 55.826925
Linescan in process: 1axis_2022-01-26_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-01-26_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-26_1030_Row3_Module1.csv
Index: 2022-01-26_1030. Wm2Front: 245.20251666666667. Wm2Back: 90.15839833333334
Linescan in process: 1axis_2022-01-26_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-01-26_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-26_1130_Row3_Module1.csv
Index: 2022-01-26_1130. Wm2Front: 677.4762666666666. Wm2Back: 292.4576666666667
Linescan in process: 1axis_2022-01-26_1230_Row3_Module1_Front
Linescan in process: 1

Linescan in process: 1axis_2022-01-29_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-29_1030_Row3_Module1.csv
Index: 2022-01-29_1030. Wm2Front: 729.8223333333333. Wm2Back: 254.29728333333333
Linescan in process: 1axis_2022-01-29_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-01-29_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-29_1130_Row3_Module1.csv
Index: 2022-01-29_1130. Wm2Front: 665.9528333333333. Wm2Back: 292.34145
Linescan in process: 1axis_2022-01-29_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-01-29_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-29_1230_Row3_Module1.csv
Index: 2022-01-29_1230. Wm2Front: 691.2022833333334. Wm2Back: 288.1162333333333
Linescan in process: 1axis_2022-01-29_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-01-29_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-01-29_1330_Row3_Module1.csv
Index: 2022-01-29_1330. Wm2Front: 782.5361666666668. Wm2Back: 235.58586666666667
Linescan in process: 1ax

In [ ]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

Linescan in process: 1axis_2022-01-01_0830_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_0830_Row3_Module25.csv
Index: 2022-01-01_0830. Wm2Front: 63.98083999999999. Wm2Back: 29.442983333333338
Linescan in process: 1axis_2022-01-01_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_0930_Row3_Module25.csv
Index: 2022-01-01_0930. Wm2Front: 68.24747. Wm2Back: 31.53105
Linescan in process: 1axis_2022-01-01_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_1030_Row3_Module25.csv
Index: 2022-01-01_1030. Wm2Front: 103.82928333333334. Wm2Back: 37.705870000000004
Linescan in process: 1axis_2022-01-01_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-01-01_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-01_1130_Row3_Module25.csv
Index: 2022-01-01_1130. Wm2Front: 262.4

Saved: results/irr_1axis_2022-01-04_1030_Row3_Module25.csv
Index: 2022-01-04_1030. Wm2Front: 625.10745. Wm2Back: 99.42043666666666
Linescan in process: 1axis_2022-01-04_1130_Row3_Module25_Front
Linescan in process: 1axis_2022-01-04_1130_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-04_1130_Row3_Module25.csv
Index: 2022-01-04_1130. Wm2Front: 589.9723. Wm2Back: 122.0858
Linescan in process: 1axis_2022-01-04_1230_Row3_Module25_Front
Linescan in process: 1axis_2022-01-04_1230_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-04_1230_Row3_Module25.csv
Index: 2022-01-04_1230. Wm2Front: 615.32075. Wm2Back: 111.74629999999999
Linescan in process: 1axis_2022-01-04_1330_Row3_Module25_Front
Linescan in process: 1axis_2022-01-04_1330_Row3_Module25_Back
Saved: results/irr_1axis_2022-01-04_1330_Row3_Module25.csv
Index: 2022-01-04_1330. Wm2Front: 680.3105. Wm2Back: 79.97135833333331
Linescan in process: 1axis_2022-01-04_1430_Row3_Module25_Front
Linescan in process: 1axis_2022-01-04_1430_Row3_

In [ ]:
demo.calculateResults()

In [ ]:
demo.CompiledResults.set_index('timestamp', inplace=True)
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

In [ ]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'A2.csv')